In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.utils import resample, shuffle

from xgboost import XGBClassifier
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='svg'
plt.rcParams.update({
    'text.usetex':False
})
import seaborn as sns

## Loading the dataset, pre-processing, and analysing the data

In [2]:
cohort_data = pd.read_csv('cohort_data_new.csv')
cohort_data

,icustay_id,anion_gap_mean,anion_gap_sd,anion_gap_min,anion_gap_max,bicarbonate_mean,bicarbonate_sd,bicarbonate_min,bicarbonate_max,calcium_total_mean,...,urea_nitrogen_min,urea_nitrogen_max,white_blood_cells_mean,white_blood_cells_sd,white_blood_cells_min,white_blood_cells_max,age,gender,icu_los_hours,target
0,217177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.700000,...,NaN,NaN,13.600000,NaN,13.6,13.6,67,M,12,1
1,219934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.200000,...,NaN,NaN,NaN,NaN,NaN,NaN,300,F,14,0
2,291702,13.000000,NaN,13.0,13.0,31.000000,NaN,31.0,31.0,7.200000,...,15.0,15.0,10.000000,NaN,10.0,10.0,45,M,28,0
3,239229,17.000000,NaN,17.0,17.0,32.000000,NaN,32.0,32.0,9.100000,...,27.0,27.0,4.800000,NaN,4.8,4.8,85,M,22,0
4,212881,17.000000,NaN,17.0,17.0,29.000000,NaN,29.0,29.0,8.800000,...,22.0,22.0,15.200000,NaN,15.2,15.2,34,F,23,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30484,268219,18.473684,3.539254,11.0,27.0,16.552632,5.717197,7.0,24.0,7.582143,...,31.0,126.0,19.989744,12.016125,6.6,45.1,70,F,596,0
30485,223087,13.428571,3.207135,11.0,18.0,28.714286,1.496026,26.0,30.0,8.514286,...,39.0,97.0,5.000000,1.272792,4.1,5.9,300,M,40,0
30486,288516,21.745455,3.378681,16.0,31.0,21.309091,2.602511,13.0,26.0,8.477083,...,34.0,146.0,15.687755,6.906357,8.3,37.7,54,M,939,0
30487,211323,18.972973,5.852286,11.0,31.0,21.621622,5.673287,8.0,33.0,9.229032,...,19.0,104.0,7.058621,2.703373,1.7,12.9,56,M,287,0


In [3]:
# Data available in most of the patients:
# This will drop the patients missing this vital information:
df = cohort_data.dropna(subset=[
    'anion_gap_mean', 'anion_gap_min', 'anion_gap_max', 'anion_gap_sd',
    'bicarbonate_mean', 'bicarbonate_min', 'bicarbonate_max', 'bicarbonate_sd',
    'calcium_total_mean', 'calcium_total_min', 'calcium_total_max', 'calcium_total_sd',
    'chloride_mean', 'chloride_min', 'chloride_max', 'chloride_sd',
    'creatinine_mean', 'creatinine_min', 'creatinine_max', 'creatinine_sd',
    'glucose_mean', 'glucose_min', 'glucose_max', 'glucose_sd',
    'hematocrit_mean', 'hematocrit_min', 'hematocrit_max', 'hematocrit_sd',
    'hemoglobin_mean', 'hemoglobin_min', 'hemoglobin_max', 'hemoglobin_sd',
    'mchc_mean', 'mchc_min', 'mchc_max', 'mchc_sd',
    'mcv_mean', 'mcv_min', 'mcv_max', 'mcv_sd',
    'magnesium_mean', 'magnesium_min', 'magnesium_max', 'magnesium_sd',
    'pt_mean', 'pt_min', 'pt_max', 'pt_sd',
    'phosphate_mean', 'phosphate_min', 'phosphate_max', 'phosphate_sd',
    'platelet_count_mean', 'platelet_count_min', 'platelet_count_max', 'platelet_count_sd',
    'potassium_mean', 'potassium_min', 'potassium_max', 'potassium_sd',
    'rdw_mean', 'rdw_min', 'rdw_max', 'rdw_sd',
    'red_blood_cells_mean', 'red_blood_cells_min', 'red_blood_cells_max', 'red_blood_cells_sd',
    'sodium_mean', 'sodium_min', 'sodium_max', 'sodium_sd',
    'urea_nitrogen_mean', 'urea_nitrogen_min', 'urea_nitrogen_max', 'urea_nitrogen_sd',
    'white_blood_cells_mean', 'white_blood_cells_min', 'white_blood_cells_max', 'white_blood_cells_sd',
    'age',  'gender', 'icu_los_hours', 'target'
    ])

# # Drop sparse columns (missing values for many ICU entries)
df = df.dropna(axis=1, how='any')
df

,icustay_id,anion_gap_mean,anion_gap_sd,anion_gap_min,anion_gap_max,bicarbonate_mean,bicarbonate_sd,bicarbonate_min,bicarbonate_max,calcium_total_mean,...,urea_nitrogen_min,urea_nitrogen_max,white_blood_cells_mean,white_blood_cells_sd,white_blood_cells_min,white_blood_cells_max,age,gender,icu_los_hours,target
400,203782,10.000000,2.828427,8.0,12.0,19.000000,0.000000,19.0,19.0,8.000000,...,35.0,35.0,14.450000,0.636396,14.0,14.9,82,M,22,0
401,228783,18.500000,0.707107,18.0,19.0,19.500000,0.707107,19.0,20.0,8.000000,...,68.0,69.0,9.350000,0.070711,9.3,9.4,72,M,27,0
402,254423,17.500000,0.707107,17.0,18.0,26.500000,0.707107,26.0,27.0,9.150000,...,41.0,44.0,9.650000,2.474874,7.9,11.4,82,F,26,0
403,227097,19.333333,2.081666,17.0,21.0,18.666667,1.154701,18.0,20.0,7.500000,...,52.0,57.0,13.000000,0.000000,13.0,13.0,300,F,39,0
404,210259,14.000000,0.000000,14.0,14.0,19.000000,0.000000,19.0,19.0,7.900000,...,22.0,23.0,12.150000,1.909188,10.8,13.5,51,F,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30483,224190,13.250000,1.258306,12.0,15.0,22.000000,2.160247,20.0,25.0,7.600000,...,26.0,44.0,9.666667,0.450925,9.2,10.1,78,M,34,0
30484,268219,18.473684,3.539254,11.0,27.0,16.552632,5.717197,7.0,24.0,7.582143,...,31.0,126.0,19.989744,12.016125,6.6,45.1,70,F,596,0
30486,288516,21.745455,3.378681,16.0,31.0,21.309091,2.602511,13.0,26.0,8.477083,...,34.0,146.0,15.687755,6.906357,8.3,37.7,54,M,939,0
30487,211323,18.972973,5.852286,11.0,31.0,21.621622,5.673287,8.0,33.0,9.229032,...,19.0,104.0,7.058621,2.703373,1.7,12.9,56,M,287,0


Summary of the filtered data

In [4]:
print(df.info())        # Data types and non-null counts
print(df.describe())    # Summary stats for numeric columns
print(df.describe(include='object')) 

<class 'pandas.core.frame.DataFrame'>
Index: 13964 entries, 400 to 30488
Data columns (total 89 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   icustay_id              13964 non-null  int64  
 1   anion_gap_mean          13964 non-null  float64
 2   anion_gap_sd            13964 non-null  float64
 3   anion_gap_min           13964 non-null  float64
 4   anion_gap_max           13964 non-null  float64
 5   bicarbonate_mean        13964 non-null  float64
 6   bicarbonate_sd          13964 non-null  float64
 7   bicarbonate_min         13964 non-null  float64
 8   bicarbonate_max         13964 non-null  float64
 9   calcium_total_mean      13964 non-null  float64
 10  calcium_total_sd        13964 non-null  float64
 11  calcium_total_min       13964 non-null  float64
 12  calcium_total_max       13964 non-null  float64
 13  chloride_mean           13964 non-null  float64
 14  chloride_sd             13964 non-null  f

In [5]:
df = df.drop('icustay_id', axis=1)
df = df.drop('gender', axis=1)
df

,anion_gap_mean,anion_gap_sd,anion_gap_min,anion_gap_max,bicarbonate_mean,bicarbonate_sd,bicarbonate_min,bicarbonate_max,calcium_total_mean,calcium_total_sd,...,urea_nitrogen_sd,urea_nitrogen_min,urea_nitrogen_max,white_blood_cells_mean,white_blood_cells_sd,white_blood_cells_min,white_blood_cells_max,age,icu_los_hours,target
400,10.000000,2.828427,8.0,12.0,19.000000,0.000000,19.0,19.0,8.000000,0.141421,...,0.000000,35.0,35.0,14.450000,0.636396,14.0,14.9,82,22,0
401,18.500000,0.707107,18.0,19.0,19.500000,0.707107,19.0,20.0,8.000000,0.000000,...,0.707107,68.0,69.0,9.350000,0.070711,9.3,9.4,72,27,0
402,17.500000,0.707107,17.0,18.0,26.500000,0.707107,26.0,27.0,9.150000,0.070711,...,2.121320,41.0,44.0,9.650000,2.474874,7.9,11.4,82,26,0
403,19.333333,2.081666,17.0,21.0,18.666667,1.154701,18.0,20.0,7.500000,0.141421,...,2.516611,52.0,57.0,13.000000,0.000000,13.0,13.0,300,39,0
404,14.000000,0.000000,14.0,14.0,19.000000,0.000000,19.0,19.0,7.900000,0.000000,...,0.707107,22.0,23.0,12.150000,1.909188,10.8,13.5,51,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30483,13.250000,1.258306,12.0,15.0,22.000000,2.160247,20.0,25.0,7.600000,0.100000,...,7.745967,26.0,44.0,9.666667,0.450925,9.2,10.1,78,34,0
30484,18.473684,3.539254,11.0,27.0,16.552632,5.717197,7.0,24.0,7.582143,0.432095,...,35.602276,31.0,126.0,19.989744,12.016125,6.6,45.1,70,596,0
30486,21.745455,3.378681,16.0,31.0,21.309091,2.602511,13.0,26.0,8.477083,0.725593,...,36.168359,34.0,146.0,15.687755,6.906357,8.3,37.7,54,939,0
30487,18.972973,5.852286,11.0,31.0,21.621622,5.673287,8.0,33.0,9.229032,1.364354,...,28.036689,19.0,104.0,7.058621,2.703373,1.7,12.9,56,287,0


In [6]:
dataset = np.array(df)
y = np.array(df['target'])
X = np.array(df.drop('target', axis=1))
print(np.shape(dataset))
print(np.shape(X))
print(np.shape(y))

# Random shuffle and split 70-30 into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))
print(f'% Readmissions in Train: {np.mean(y_train) * 100}')
print(f'% Readmissions in Test: {np.mean(y_test) * 100}')

(13964, 87)
(13964, 86)
(13964,)
(9774, 86)
(4190, 86)
(9774,)
(4190,)
% Readmissions in Train: 14.027010435850215
% Readmissions in Test: 11.957040572792362


### XGBoost 

In [7]:
import numpy as np
import optuna
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

RANDOM_STATE = 229
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)


def objective(trial):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "use_label_encoder": False,
        "random_state": RANDOM_STATE,
        "n_jobs": -1,

        # Search around your known good values
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "max_delta_step": trial.suggest_int("max_delta_step", 0, 5),
        "max_leaves": trial.suggest_int("max_leaves", 2, 10),
        "min_child_weight": trial.suggest_float("min_child_weight", 1, 8),
        "n_estimators": trial.suggest_int("n_estimators", 400, 1000),
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "lambda": trial.suggest_float("lambda", 0.8, 1.5),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.5, 1.5),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
    }

    aucs = []
    for train_idx, valid_idx in cv.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[valid_idx]
        y_tr, y_val = y_train[train_idx], y_train[valid_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        y_pred = model.predict_proba(X_val)[:, 1]
        aucs.append(roc_auc_score(y_val, y_pred))

    return np.mean(aucs)

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\nBest Parameters Found:")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

print("\nBest Cross-Validation AUC:", study.best_value)

best_params = study.best_params
best_params.update({
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "use_label_encoder": False,
    "random_state": RANDOM_STATE,
    "n_jobs": -1,
})

final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

[I 2025-11-12 13:16:43,138] A new study created in memory with name: no-name-8684da84-e968-4e99-a7b9-29b07317fc64


  0%|          | 0/50 [00:00<?, ?it/s]

c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:16:43] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:16:44] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:16:45] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:16:45] WARNING: C:\b\abs_d9

[I 2025-11-12 13:16:52,688] Trial 0 finished with value: 0.6774409922224773 and parameters: {'learning_rate': 0.01250331001543945, 'max_depth': 10, 'max_delta_step': 4, 'max_leaves': 4, 'min_child_weight': 6.456232181430629, 'n_estimators': 682, 'alpha': 0.4346791939075747, 'lambda': 1.2687132500980753, 'scale_pos_weight': 0.8541608132425591, 'subsample': 0.9897997300994384}. Best is trial 0 with value: 0.6774409922224773.


c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:16:52] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:16:54] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:16:55] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:16:57] WARNING: C:\b\abs_d9

...

[I 2025-11-12 13:41:26,788] Trial 48 finished with value: 0.6868682892816413 and parameters: {'learning_rate': 0.023341657437502817, 'max_depth': 6, 'max_delta_step': 3, 'max_leaves': 6, 'min_child_weight': 6.152963988217847, 'n_estimators': 924, 'alpha': 0.21854845527758182, 'lambda': 0.9067249873330223, 'scale_pos_weight': 0.5002979086513041, 'subsample': 0.8406073338774318}. Best is trial 12 with value: 0.6883316640778888.


c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:41:26] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:41:30] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:41:34] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:41:38] WARNING: C:\b\abs_d9

[I 2025-11-12 13:42:02,888] Trial 49 finished with value: 0.682428232468151 and parameters: {'learning_rate': 0.03966869019085301, 'max_depth': 4, 'max_delta_step': 4, 'max_leaves': 6, 'min_child_weight': 1.0620338327624217, 'n_estimators': 768, 'alpha': 0.9948334000945239, 'lambda': 1.4234058199872266, 'scale_pos_weight': 1.3971750161062575, 'subsample': 0.8935243460986789}. Best is trial 12 with value: 0.6883316640778888.

Best Parameters Found:
learning_rate: 0.027458484360054512
max_depth: 4
max_delta_step: 3
max_leaves: 6
min_child_weight: 7.946284330625193
n_estimators: 1000
alpha: 0.10740365418972755
lambda: 1.4796609022621676
scale_pos_weight: 0.5104372886753539
subsample: 0.7733951221229483

Best Cross-Validation AUC: 0.6883316640778888


c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:42:02] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(alpha=0.10740365418972755, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              lambda=1.4796609022621676, learning_rate=0.027458484360054512,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=3, max_depth=4, max_leaves=6,
              min_child_weight=7.946284330625193, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1000, ...)

In [8]:
y_proba_test = final_model.predict_proba(X_test)[:, 1]
y_pred_test = (y_proba_test >= 0.5).astype(int)

test_auroc = roc_auc_score(y_test, y_proba_test)
print(f"\nFinal Test ROC AUC: {test_auroc:.4f}")


Final Test ROC AUC: 0.6856
